In [236]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.options.display.float_format = '{:.2f}'.format

def gerarDataSet2014():
    
    arquivos = []
    for x in range(1, 13):
        try:
            nome_arq = "ReceitasEstaduais_"+"20140"+str(x)+".csv"
            arquivo = pd.read_csv(str(nome_arq), sep=',', encoding='ISO-8859-1', dtype='str')
            arquivos.append(arquivo)
            
        except FileNotFoundError:
            nome_arq = "ReceitasEstaduais_"+"2014"+str(x)+".csv"
            arquivo = pd.read_csv(str(nome_arq), sep=',', encoding='ISO-8859-1', dtype='str')
            arquivos.append(arquivo)

    dataset = []
    for pos in range(0, 12):
        dataset.append(arquivos[pos])
        
    return dataset

def gerarDataSet2015():
    arquivos = []
    for x in range(1, 9):
        try:
            nome_arq = "ReceitasEstaduais_"+"20150"+str(x)+".csv"
            arquivo = pd.read_csv(str(nome_arq), sep=',', encoding='ISO-8859-1', dtype='str')
            arquivos.append(arquivo)
            
        except FileNotFoundError as erro:
            print("Erro: ",erro)
            
    dataset = []
    for pos in range(0, 8):
        dataset.append(arquivos[pos])
        
    return dataset       
   
frame_2014 = []   
for pos in range(0, 12):
    df = pd.DataFrame(gerarDataSet2014()[pos])
    frame_2014.append(df)

union2014 = pd.concat(frame_2014)

frame_2015 = []
for pos in range(0, 8):
    df = pd.DataFrame(gerarDataSet2015()[pos])
    frame_2015.append(df)
    
union2015 = pd.concat(frame_2015)

union_all = pd.concat([union2014, union2015])

ANOMES = []
CNPJORGAO = []
NOMEORGAO = []
RECEITAPREVISTAATUALIZADA = []
RECEITAREALIZADA = []
TIPOADMINISTRACAO = []
TIPOPODER = []

for ano in union_all['ANOMES']:
    ANOMES.append(ano)
for cnpj in union_all['CNPJORGAO']:
    CNPJORGAO.append(cnpj)
for orgao in union_all['NOMEORGAO']:
    NOMEORGAO.append(orgao)
for ra in union_all['RECEITAPREVISTAATUALIZADA']:
    RECEITAPREVISTAATUALIZADA.append(ra)
for rr in union_all['RECEITAREALIZADA']:
    RECEITAREALIZADA.append(rr)
for ta in union_all['TIPOADMINISTRACAO']:
    TIPOADMINISTRACAO.append(ta)
for tp in union_all['TIPOPODER']:
    TIPOPODER.append(tp)
    
dataset = {'DATA':ANOMES,
           'CNPJ':CNPJORGAO,
           'ORGARO':NOMEORGAO,
           'RECEITAATUALIZADA':RECEITAPREVISTAATUALIZADA,
           'RECEITAREALIZADA':RECEITAREALIZADA,
           'TIPOADMINISTRACAO':TIPOADMINISTRACAO,
           'TIPOPODER':TIPOPODER}
    
df_principal = pd.DataFrame(dataset)

def tratarColunaCNPJORGAO(row):
    if len(str(row)) == 13:
        return "0"+str(row)
    elif str(row) == 'nan':
        return "-"
    else:
        return row

def tratarColunaRECEITAS(row):
    if str(row) == 'nan':
        return "0"
    else:
        return format(float(row), ".15g")
    
def tratarColunaANOMES(row):
    ano = row[:4]
    mes = row[4:6]
    minha_data = "1"+"/"+mes+"/"+ano
    date = datetime.strptime(minha_data, '%d/%m/%Y')
           
    return date

df_principal['RECEITAATUALIZADA'] = df_principal['RECEITAATUALIZADA'].apply(tratarColunaRECEITAS)
df_principal['RECEITAREALIZADA'] = df_principal['RECEITAREALIZADA'].apply(tratarColunaRECEITAS)
df_principal['CNPJ'] = df_principal['CNPJ'].apply(tratarColunaCNPJORGAO)  
df_principal['DATA'] = df_principal['DATA'].apply(tratarColunaANOMES)

class gerarAnalises():
    
    def __init__(self, cnpj, data, orgao, recetprev, recetreal):
        self.cnpj = cnpj
        self.data = data
        self.orgao = orgao
        self.recetprev = recetprev
        self.recetreal = recetreal
        
        recetprev_f = []
        for i in self.recetprev:
            recetprev_f.append(float(i))
            
        recetreal_f = []
        for i in self.recetreal:
            recetreal_f.append(float(i))
        
        dataset = {'cnpj':self.cnpj,
                   'data':self.data,
                   'orgao':self.orgao,
                   'receitaprevia':recetprev_f,
                   'receitarecetreal':recetreal_f}
        
        df = pd.DataFrame(dataset)
        self.df = df
        
    def DistribuicaoReceitaMensal(self):

        imput_grps = self.df.pivot_table(values=['receitaprevia', 'receitarecetreal'], index=['data'], 
                                           aggfunc=np.sum, 
                                           dropna=False)        
        percents = []
        datas = []
        receitaprevista = []
        receitareal = []
        for i in range(0, imput_grps.shape[0]):
            data = str(imput_grps.index[i]).split("-")[1]+"-"+str(imput_grps.index[i]).split("-")[0]
            datas.append(data)
            receitaprevista.append(imput_grps['receitaprevia'][i])
            receitareal.append(imput_grps['receitarecetreal'][i])
            percent = (imput_grps['receitarecetreal'][i]/imput_grps['receitaprevia'][i])*100
            percents.append(percent)
            
        df_mes_receita = pd.DataFrame({'data':datas,
                                       'Receita Prevista':receitaprevista,
                                       'Receita Realizada':receitareal,
                                       'Porcentagem':percents}, columns = ['data',
                                                                           'Receita Prevista',
                                                                           'Receita Realizada',
                                                                           'Porcentagem'])   
            
        return df_mes_receita
    
    def maiorReceitaRealizada(self):
        
        return self.df.loc[self.df['receitaprevia'].idxmax()]
    
    def relacaoDeReceitaPorOrgao(self):
        
        imput_grps = self.df[self.df['receitarecetreal'] != 0].pivot_table(values = ['receitarecetreal'], 
                                          index = ['orgao', 'data'],
                                          aggfunc = np.sum,
                                          dropna=False)
        
        return imput_grps
    
    def mediaGasto(self):
        print("Descrição geral:")
        print(self.df['receitarecetreal'].describe())
        
        imput_grps = self.df[self.df['receitarecetreal'] != 0].pivot_table(values = ['receitarecetreal'], 
                                          index = ['data'],
                                          aggfunc = np.mean,
                                          dropna=False)
        print("Média distribuida mensalmente: ")
        print(imput_grps)

obj = gerarAnalises(df_principal['CNPJ'], 
                    df_principal['DATA'], 
                    df_principal['ORGARO'], 
                    df_principal['RECEITAATUALIZADA'], 
                    df_principal['RECEITAREALIZADA'])

print("Percentual de receita realizada em relação a prevista:")
print(obj.DistribuicaoReceitaMensal())
print("\nMaior receita realizada:")
print(obj.maiorReceitaRealizada())
print("\nDescrição estatística da média de gasto mensal:")
print(obj.mediaGasto())
print("\nSoma de receita por órgão:")
obj.relacaoDeReceitaPorOrgao()


Percentual de receita realizada em relação a prevista:
       data  Receita Prevista  Receita Realizada  Porcentagem
0   01-2014    41108390663.26     35056655560.49        85.28
1   02-2014    41108390663.26     16391558895.22        39.87
2   03-2014    41108390663.26     14958529432.87        36.39
3   04-2014    41108390663.26     22298772982.16        54.24
4   05-2014    41108390663.26     17763845540.31        43.21
5   06-2014    41108390663.26     17588998152.41        42.79
6   07-2014    41108390663.26     18027128882.96        43.85
7   08-2014    41108390663.26     17255550005.30        41.98
8   09-2014    41108390663.26     22646662939.28        55.09
9   10-2014    41108390663.26     20662834500.32        50.26
10  11-2014    41108390663.26     19937870243.70        48.50
11  12-2014    41108390704.14               0.00         0.00
12  01-2015    49079440327.78               0.00         0.00
13  02-2015    49079440327.78               0.00         0.00
14  03-2015    

receitarecetreal
orgao                                    data                        
AG. GOIANA DE DEFESA AGROPECUARIA        2014-01-01       13682753.92
                                         2014-02-01        5612894.43
                                         2014-03-01        4819038.28
                                         2014-04-01        5227683.69
                                         2014-05-01        4743781.95
                                         2014-06-01        5198333.49
                                         2014-07-01        5101012.50
                                         2014-08-01        4117927.86
                                         2014-09-01        4427141.46
                                         2014-10-01        5005530.15
                                         2014-11-01        3397092.51
                                         2015-03-01               nan
                                         2015-04-01               nan
                                         2015-05-01               nan
AG. GOIANA DE REG. CONT. E FISC. S.PUBL  2014-01-01        5261548.30
                                         2014-02-01        3844000.79
                                         2014-03-01        3414226.58
                                         2014-04-01        4733328.02
                                         2014-05-01        4023578.87
                                         2014-06-01        2807558.99
                                         2014-07-01        3066327.69
                                         2014-08-01        3444458.33
                                         2014-09-01        3142375.34
                                         2014-10-01        2895188.77
                                         2014-11-01        2769310.30
                                         2015-03-01        5027970.34
                                         2015-04-01        5418105.71
                                         2015-05-01        4959752.75
AG. GOIANA DE TRANSP. E OBRAS PUBLICAS   2014-01-01       48286986.86
                                         2014-02-01       23045817.94
...                                                               ...
SECRETARIA DE GESTÃO E PLANEJAMENTO     2015-04-01          30882.20
                                         2015-05-01          25049.25
UNIVERSIDADE ESTADUAL DE GOIAS           2014-01-01         774954.77
                                         2014-02-01        1601681.70
                                         2014-03-01        2623517.42
                                         2014-04-01        7398982.76
                                         2014-05-01        3585565.62
                                         2014-06-01         415515.91
                                         2014-07-01        9786707.32
                                         2014-08-01         662788.63
                                         2014-09-01        1567533.64
                                         2014-10-01        9435993.83
                                         2014-11-01        4138304.34
                                         2015-03-01               nan
                                         2015-04-01               nan
                                         2015-05-01               nan
UNIVERSIDADE ESTADUAL DE GOIÃS - UEG Â  2014-01-01               nan
                                         2014-02-01               nan
                                         2014-03-01               nan
                                         2014-04-01               nan
                                         2014-05-01               nan
                                         2014-06-01               nan
                                         2014-07-01               nan
                                         2014-08-01               nan
                                         2014-09-01            

In [217]:

df_principal

,CNPJ,DATA,ORGARO,RECEITAATUALIZADA,RECEITAREALIZADA,TIPOADMINISTRACAO,TIPOPODER
0,04354889000166,2014-01-01,FUNDO SOCIAL DE LOTERIA,29840416.67,0,Direta,Executivo
1,06084202000145,2014-01-01,PROTEGE GOIAS,29840416.67,26710912.46,Direta,Executivo
2,-,2014-01-01,ESTADO DE GOIAS,34422000,0,NaN,NaN
3,-,2014-01-01,ESTADO DE GOIAS - FUNDOS ESPECIAIS,251916.66,0,NaN,NaN
4,-,2014-01-01,ESTADO DE GOIAS - PODER EXECUTIVO,34421000,0,NaN,NaN
5,-,2014-01-01,ESTADO DE GOIAS-ADMINISTRACAO INDIRETA,166.66,0,NaN,NaN
6,01112580000171,2014-01-01,UNIVERSIDADE ESTADUAL DE GOIAS,83.33,0,Indireta,Executivo
7,-,2014-01-01,ESTADO DE GOIAS - ADMINISTRACAO DIRETA,250,0,NaN,NaN
8,13301077000143,2014-01-01,FUNDO CAP. SERV. E MOD. - FUNCAM,238833.33,0,Direta,Executivo
9,04916251000171,2014-01-01,FUNDO ESTADUAL DE DESENVOL.DO NORDESTE,238833.33,0,Direta,Executivo
